In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df_train = pd.read_csv(r"C:\Users\91896\Smart_bridge_Project\dreaddit-train.csv")
df_train.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [3]:
df_test = pd.read_csv(r"C:\Users\91896\Smart_bridge_Project\dreaddit-test.csv")
df_test.head()

,id,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,896,relationships,7nu7as,"[50, 55]","Its like that, if you want or not.“ ME: I have...",0,0.8,1.514981e+09,22,-1.238793,...,1.0000,1.2000,1.0,1.65864,1.32245,1.80264,0.63,62,-0.148707,0.000000
1,19059,anxiety,680i6d,"(5, 10)",I man the front desk and my title is HR Custom...,0,1.0,1.493348e+09,5,7.684583,...,1.4000,1.1250,1.0,1.69133,1.69180,1.97249,1.00,2,7.398222,-0.065909
2,7977,ptsd,8eeu1t,"(5, 10)",We'd be saving so much money with this new hou...,1,1.0,1.524517e+09,10,2.360408,...,1.1429,1.0000,1.0,1.70974,1.52985,1.86108,1.00,8,3.149288,-0.036818
3,1214,ptsd,8d28vu,"[2, 7]","My ex used to shoot back with ""Do you want me ...",1,0.5,1.524018e+09,5,5.997000,...,1.0000,1.3000,1.0,1.72615,1.52000,1.84909,1.00,7,6.606000,-0.066667
4,1965,relationships,7r1e85,"[23, 28]",I haven’t said anything to him yet because I’m...,0,0.8,1.516200e+09,138,4.649418,...,1.1250,1.1429,1.0,1.75642,1.43582,1.91725,0.84,70,4.801869,0.141667


In [4]:
df_final = pd.concat([df_train,df_test])
df_final

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.800000,1.521614e+09,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.000000,1.527010e+09,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.800000,1.535936e+09,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.600000,1.516430e+09,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.800000,1.539809e+09,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,ptsd,7vavh4,"(0, 5)",i have horrible vivid nightmares every night. ...,24271,1,1.000000,1.517790e+09,7,4.597368,...,1.000,1.2000,1.0,1.76956,1.46522,1.84112,0.79,15,4.589333,-0.075000
711,relationships,7oeyuc,"[70, 75]",Also I can't think about both of them without ...,2212,1,0.571429,1.515192e+09,0,1.452083,...,1.000,1.1818,1.0,1.67960,1.51429,1.83530,0.50,7,2.833889,0.189286
712,relationships,7ssxec,"(35, 40)","Furthermore, I told him before we got really s...",27951,1,0.600000,1.516849e+09,3,3.956447,...,1.000,1.0000,1.0,1.71367,1.50909,1.85501,0.72,10,5.709053,-0.044444
713,assistance,90bila,"[8, 13]",Here's the link to my amazon wish list where t...,1164,0,0.600000,1.532048e+09,18,0.236814,...,1.500,1.0000,1.0,1.65132,1.53913,1.90766,0.74,2,2.365294,0.090000


In [5]:
df_final = df_final[["text","label"]]
df_final.sample(10)

,text,label
2201,"He says she is a great girl, caring for him an...",0
2501,"So, a long time ago when I was in around the 2...",0
1521,In order to make a case for its commercializat...,0
2168,I want that. I want him as well as a SO but ma...,0
2602,"Not actually losing my home at the moment, so ...",1
1514,Once I committed mentally to therapy and knew ...,0
2237,Both him and my mom left pornographic magazine...,0
338,We wish to teach such kids and train their fam...,0
1839,I'm building a school PC. My current laptop la...,0
2092,"Hello, You are invited to complete a survey fo...",0


In [6]:
def pre_porccess(text):
    text = str(text).lower()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [7]:
df_final["text"] = df_final["text"].apply(pre_porccess)
df_final

C:\Users\91896\AppData\Local\Temp\ipykernel_2372\2913819218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["text"] = df_final["text"].apply(pre_porccess)


,text,label
0,"he said he had not felt that way before, sugge...",1
1,"hey there r/assistance, not sure if this is th...",0
2,my mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,october is domestic violence awareness month a...,1
...,...,...
710,i have horrible vivid nightmares every night. ...,1
711,also i can't think about both of them without ...,1
712,"furthermore, i told him before we got really s...",1
713,here's the link to my amazon wish list where t...,0


In [8]:
def Root_wort(text):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    
    text = [word for word in text.split(' ') if word not in stopwords]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

In [9]:
df_final["text"] = df_final["text"].apply(Root_wort)
df_final['text']

C:\Users\91896\AppData\Local\Temp\ipykernel_2372\3599492805.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["text"] = df_final["text"].apply(Root_wort)


0      said felt way before, sugget go rest ..trigger...
1      hey r/assistance, sure right place post this.....
2      mom hit newspap shock would this, know like pl...
3      met new boyfriend, amazing, kind, sweet, good ...
4      octob domest violenc awar month domest violenc...
                             ...                        
710    horribl vivid nightmar everi night. sometim th...
711    also can't think without get angri jealou agai...
712    furthermore, told got realli seriou anxieti de...
713    here' link amazon wish list two item are:  lin...
714    keep us protected? alreadi told unwelcom perso...
Name: text, Length: 3553, dtype: object

In [10]:
x = df_final["text"]
y = df_final["label"]

In [11]:
# Set Pandas display options to show full content of columns
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

In [12]:
print(x.iloc[0])

said felt way before, sugget go rest ..trigger ahead youi'r hypocondriac like me: decid look "feel doom" hope mayb get suck rabbit hole ludicr conspiracy, stupid "are psychic" test new age b.s., someth could even laugh road. no, end read sens doom indic variou health ailments; one prone to.. top "doom" gloom..i f'n worri heart. happen physic  hours.


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(x)

In [14]:
#tfidf_array = tfidf_matrix.toarray()
#tfidf_matrix
#tfidf_array

In [33]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tfidf_matrix,y,test_size=0.2,random_state=42)

In [34]:
from sklearn.linear_model import LogisticRegression
m2=LogisticRegression().fit(x_train, y_train)
score = m2.score(x_test,y_test)
#accuracy_score(m2,y_test)
print("Model Accuracy is {p}%".format(p =round (score*100, 2)))

Model Accuracy is 74.82%


In [40]:
import pickle
model_data = {'vectorizer': tfidf_vectorizer, 'model': m2}
with open('my_model.pkl', 'wb') as model_file:
    pickle.dump(model_data, model_file)

In [39]:
# Using Naive Bayes
#prompt = "The weather is pleasant"
#p = tfidf_vectorizer.transform([prompt]).toarray()
#op = m2.predict(p)
#print(op)

[0]
